In [1]:
import geopandas as gpd
import json
import pandas as pd

In [2]:
import numpy as np

In [3]:
import folium

In [4]:
test = gpd.read_file('/Volumes/data/zillow-neighborhoods_real.geojson')

In [16]:
test.head()

,city,name,regionid,county,state,geometry
0,Chicago,Dunning,138129,Cook,IL,"POLYGON ((-87.78618999999989 41.94558, -87.786..."
1,Danville,Vermilipn Heights,140779,Vermilion,IL,POLYGON ((-87.65849448899995 40.12343103000006...
2,Springfield,Jefferson Park,195637,Sangamon,IL,"POLYGON ((-89.700805 39.812098, -89.696901 39...."
3,Chicago,Gresham,269571,Cook,IL,"POLYGON ((-87.6337397026425 41.72996999541133,..."
4,Chicago,South Shore,269608,Cook,IL,"POLYGON ((-87.56564699999987 41.774134, -87.56..."


In [5]:
chicago = test.loc[test['city'] == 'Chicago']

In [17]:
chicago.head()

,city,name,regionid,county,state,geometry,stat
0,Chicago,Dunning,138129,Cook,IL,"POLYGON ((-87.78618999999989 41.94558, -87.786...",0.649962
1,Chicago,Gresham,269571,Cook,IL,"POLYGON ((-87.6337397026425 41.72996999541133,...",0.458980
2,Chicago,South Shore,269608,Cook,IL,"POLYGON ((-87.56564699999987 41.774134, -87.56...",0.170977
3,Chicago,West Lawn,269614,Cook,IL,POLYGON ((-87.71284691599992 41.78508602300007...,0.411776
4,Chicago,Beverly,272951,Cook,IL,POLYGON ((-87.67797391517639 41.74480002257391...,0.662531


In [6]:
sampl = np.random.uniform(low=0.001, high=.95, size=(228,))

In [7]:
chicago['stat'] = sampl

/Users/debrastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
chicago.reset_index(drop=True,inplace=True)
chi_json = chicago.to_json()

In [9]:
geo_json_data = json.loads(chi_json)

In [10]:
chicago_tidy = pd.DataFrame(chicago)[['regionid', 'stat']]

In [11]:
m = folium.Map([41.8, -87.5], zoom_start=10)

chloropleth = folium.Choropleth(
    geo_data=chicago,
    data=chicago_tidy,
    columns=['regionid', 'stat'],
    key_on='feature.properties.regionid',
    fill_color='YlGnBu',
    highlight=True,
    name='Neighborhood Affluency',
    #bins=[3, 4, 9, 11]
).add_to(m)

m

In [15]:
m = folium.Map([41.8, -87.5], zoom_start=10)

folium.GeoJson(
    geo_json_data,
    style_function=lambda x :{'fillColor':'red' if \
      float(x['properties']['stat'])< .25 \
       else 'orange' if float(x['properties']['stat'])< .5 \
        else 'yellow' if float(x['properties']['stat'])< .75 \
        else 'green', 
        'color': 'black',
        'weight': 2,
        'dashArray': '5,5'
        }
).add_to(m)

#m.save(os.path.join('results', 'GeoJSON_and_choropleth_4.html'))

m